# Школа алготрейдеров. Блок торгового ПО и программирования
## Занятие 7. Работа с базами данных SQLite. Написание простейших индикаторов

### База данных SQLite, модуль `sqlite3`

* [Сайт SQLite](https://www.sqlite.org/index.html)
* [Модуль `sqlite3` для Python](https://docs.python.org/3.6/library/sqlite3.html)
* [Модуль `pandas.io.sql`](http://pandas.pydata.org/pandas-docs/stable/io.html#sql-queries) (абстракция над SQL)

In [13]:
import sqlite3

#### Создание нового файла с базой данных

In [16]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

#### Создание новой таблицы

In [17]:
c.execute('''
    create table Stocks (
        Ticker text,
        Date text,
        Open double,
        High double,
        Low double,
        Close double,
        primary key (Ticker, Date)
    )
''')

#### Заполнение таблицы

In [18]:
c.execute('''
    insert into Stocks (
        Ticker, Date,
        Open, High,
        Low, Close
    )
    values (
        'TEST', '20161013',
        6, 10, 4.2, 8
    )
''')

In [19]:
rows = [
    ('MANY', '20161014', 40.5, 51.2, 40.2, 43.4),
    ('MANY', '20161015', 43.9, 54.1, 42.7, 45.4),
    ('MANY', '20161016', 45.5, 49.0, 45.1, 45.2),
]
c.executemany('''
    insert into Stocks (
        Ticker, Date,
        Open, High,
        Low, Close
    )
    values (?, ?, ?, ?, ?, ?)
''', rows)

In [20]:
conn.commit()

#### Заполнение таблицы данными из `pandas.DataFrame`

In [21]:
from pandas_datareader import data
import datetime as dt
fb = data.DataReader('fb', 'yahoo', dt.datetime(2012, 6, 1), dt.datetime(2016, 8, 30))
fb.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2012-06-01,28.889999,29.15,27.389999,27.719999,41855500,27.719999
2012-06-04,27.200001,27.65,26.440001,26.900000,35230300,26.900000
2012-06-05,26.700001,27.76,25.750000,25.870001,42473400,25.870001
2012-06-06,26.070000,27.17,25.520000,26.809999,61489200,26.809999
2012-06-07,27.000000,27.35,26.150000,26.309999,26159500,26.309999


In [41]:
data = [
    ('FB', row.name.strftime('%Y%m%d'), row.Open, row.High, row.Low, row.Close)
    for i, row in fb.iterrows()
]
data[:2]

[('FB',
  '20120601',
  28.889999,
  29.149999999999999,
  27.389999,
  27.719998999999998),
 ('FB',
  '20120604',
  27.200001,
  27.649999999999999,
  26.440001000000002,
  26.899999999999999)]

In [42]:
c.executemany('''
    insert into Stocks (
        Ticker, Date,
        Open, High,
        Low, Close
    )
    values (?, ?, ?, ?, ?, ?)
''', data)

In [43]:
conn.commit()

#### Загрузка данных

In [46]:
c.execute('select * from Stocks')

In [52]:
c.fetchone()

('FB', '20120604', 27.200001, 27.65, 26.440001000000002, 26.9)

In [53]:
list(c.execute('select * from Stocks'))[:10]

[('TEST', '20161013', 6.0, 10.0, 4.2, 8.0),
 ('MANY', '20161014', 40.5, 51.2, 40.2, 43.4),
 ('MANY', '20161015', 43.9, 54.1, 42.7, 45.4),
 ('MANY', '20161016', 45.5, 49.0, 45.1, 45.2),
 ('FB', '20120601', 28.889999, 29.15, 27.389999, 27.719998999999998),
 ('FB', '20120604', 27.200001, 27.65, 26.440001000000002, 26.9),
 ('FB', '20120605', 26.700001, 27.76, 25.75, 25.870001000000002),
 ('FB', '20120606', 26.07, 27.17, 25.52, 26.809998999999998),
 ('FB', '20120607', 27.0, 27.35, 26.15, 26.309998999999998),
 ('FB', '20120608', 26.549999, 27.76, 26.440001000000002, 27.1)]

In [54]:
list(c.execute('''
    select
        Ticker,
        avg(Open),
        avg(Close),
        min(Low),
        max(High)
    from
        Stocks
    group by
        Ticker
'''))

[('FB', 66.52301222263792, 66.51880258746486, 17.549999, 128.330002),
 ('MANY', 43.300000000000004, 44.666666666666664, 40.2, 54.1),
 ('TEST', 6.0, 8.0, 4.2, 10.0)]

In [57]:
c.execute('''
    select
        Ticker,
        avg(Open),
        avg(Close),
        min(Low),
        max(High)
    from
        Stocks
    group by
        Ticker
''')

for ticker, avg_open, avg_close, low, high in c:
    print('Range for {}: [{}, {}]'.format(ticker, low, high))

Range for FB: [17.549999, 128.330002]
Range for MANY: [40.2, 54.1]
Range for TEST: [4.2, 10.0]


#### Закрытие соединения

In [12]:
conn.close()

#### Консольный клиент

Запуск из командной строки: `sqlite3 example.db`

#### `.tables`, `.schema`

#### `.headers on`, `.mode column`

#### Type affinity

Нет строгих ограничений на тип

In [61]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

In [62]:
c.execute('''
    insert into Stocks (
        Ticker, Date,
        Open, High,
        Low, Close
    )
    values (
        ?, ?, ?, ?, ?, ?
    )
''', (20160101, 81, 'a', 'b', 'c', 'd'))

In [63]:
conn.commit()

In [64]:
list(c.execute('select * from Stocks where Ticker = 20160101'))

[('20160101', '81', 'a', 'b', 'c', 'd')]

In [65]:
conn.close()

### [Система Quanteon](http://quanteon.ru)